# Итоговый проект по курсу "ML для финансового анализа"

*Автор - Карданов М. Т.*

## Задача проекта:

Разработать торгового робота, представляющего из себя ансамбль моделей технического анализа и машинного обучения, анализирующего данные о стоимости акций компании Google за последний год и формулирующего торговые действия.

## План проекта:

1. Загрузка данных;
1. Feature engineering;
1. Построение модели технического анализа;
1. Построение модели машинного обучения;
1. Обучение ансамбля;
1. Проверка качества ансамбля на тестовой выборке;
1. Составление презентации, защита.

## 0: Импорты

In [51]:
import pandas as pd
import sklearn
import numpy as np
import yfinance
import os
import warnings

## 1: Загрузка данных

In [52]:
def load_data() -> pd.DataFrame:

    data = yfinance.download(tickers="GOOG", 
                             period="5y", 
                             interval="1d", 
                             multi_level_index=False)
    data.index = pd.to_datetime(data.index)

    return data

## 2: Feature-engineering

In [53]:
def add_date_features(data: pd.DataFrame) -> pd.DataFrame:

    data["Date_time"] = data.index

    data['Year']=data['Date_time'].dt.strftime('%Y')
    data['Month']=data['Date_time'].dt.strftime('%m')
    data['Day']=data['Date_time'].dt.strftime('%d')

    data = data.drop("Date_time", axis=1)

    return data

В качестве признаков технического анализа я решил выбрать 5-ти, 10-ти и 20-барные MA и EMA, в основном из-за того, что они просты в реализации и, одновременно с этим, несут в себе много информации о более долгосрочном поведении цены.

In [54]:
def add_price_features(data: pd.DataFrame) -> pd.DataFrame:

    data["5_day_MA"] = data["Close"].rolling(window=5, min_periods=1).mean()
    data["10_day_MA"] = data["Close"].rolling(window=10, min_periods=1).mean()
    data["20_day_MA"] = data["Close"].rolling(window=20, min_periods=1).mean()

    data["5_day_EMA"] = data["Close"].ewm(span=5, min_periods=1).mean()
    data["10_day_EMA"] = data["Close"].ewm(span=10, min_periods=1).mean()
    data["20_day_EMA"] = data["Close"].ewm(span=20, min_periods=1).mean()

    return data

In [55]:
def save_data(data: pd.DataFrame, filename: str = "data/processed_data.csv") -> None:

    if os.path.exists(filename):
        data.to_csv(filename, mode='a', header=False, index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [56]:
def temp_load():
    data = load_data()
    data = add_date_features(data)
    data = add_price_features(data)
    save_data(data)

In [57]:
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore")
    temp_load()

[*********************100%***********************]  1 of 1 completed
